In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/nn_adv/class_01/Laborat%C3%B3rios/lab_utils_ml_adv_week_1.zip
      
!unzip -n -q lab_utils_ml_adv_week_1.zip

In [ ]:
# Testar se estamos no Google Colab
try:
  import google.colab
  IN_COLAB = True
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  IN_COLAB = False

# Neurônios e Camadas
Neste laboratório, exploraremos o funcionamento interno dos neurônios/unidades e camadas. Em particular, o laboratório estabelecerá paralelos com os modelos que você viu anteriormente, o modelo de regressão/linear e o modelo logístico. O laboratório apresentará o Tensorflow e demonstrará como esses modelos são implementados nessa estrutura.<figure>
   <img src="./images/C2_W1_NeuronsAndLayers.png"  style="width:540px;height:200px;" >
</figure>


## Pacotes
**Tensorflow e Keras**  
O Tensorflow é um pacote de aprendizado de máquina desenvolvido pelo Google. Em 2019, o Google integrou o Keras ao Tensorflow e lançou o Tensorflow 2.0. O Keras é uma estrutura desenvolvida de forma independente por François Chollet que cria uma interface simples e centrada em camadas para o Tensorflow. Este curso usará a interface do Keras.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy
from tensorflow.keras.activations import sigmoid
from lab_utils_common import dlc
from lab_neurons_utils import plt_prob_1d, sigmoidnp, plt_linear, plt_logistic
plt.style.use('./deeplearning.mplstyle')
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

## Neurônios sem ativação - Modelo de Regressão/Linear

### Conjunto deDados
Usaremos um exemplo do visto anteriormente: a regressão linear dos preços de imóveis.

In [ ]:
X_train = np.array([[1.0], [2.0]], dtype=np.float32)           #(tamanha em 1000 de pés quadrados)
Y_train = np.array([[300.0], [500.0]], dtype=np.float32)       #(preço em 1000s de dólares)

fig, ax = plt.subplots(1,1)
ax.scatter(X_train, Y_train, marker='x', c='r', label="Pontos de Dados")
ax.legend( fontsize='xx-large')
ax.set_ylabel('Preço (em 1000s de dólares)', fontsize='xx-large')
ax.set_xlabel('Tamanho (1000 sqft)', fontsize='xx-large')
plt.show()

### Modelo de Regressão/Linear
A função implementada por um neurônio sem ativação é a regressão linear vista anteriormente:
$$ f_{\mathbf{w},b}(x^{(i)}) = \mathbf{w}\cdot x^{(i)} + b \tag{1}$$



Podemos definir uma camada com um neurônio ou unidade e compará-la com a conhecida função de regressão linear.

In [ ]:
linear_layer = tf.keras.layers.Dense(units=1, activation = 'linear', )

Vamos examinar os pesos.

In [ ]:
linear_layer.get_weights()

Não há pesos, pois os pesos ainda não foram instanciados. Vamos testar o modelo em um exemplo em `X_train`. Isso acionará a instanciação dos pesos. Observe que a entrada para a camada deve ser 2-D, portanto, vamos reformulá-la.

In [ ]:
a1 = linear_layer(X_train[0].reshape(1,1))
print(a1)

O resultado é um tensor (outro nome para uma matriz) com uma forma de (1,1) ou uma entrada.   
Agora vamos dar uma olhada nos pesos e no bias . Esses pesos são inicializados aleatoriamente com números pequenos e o bias tem como padrão ser inicializada com zero.

In [ ]:
w, b= linear_layer.get_weights()
print(f"w = {w}, b={b}")

Um modelo de regressão linear (1) com um único recurso de entrada terá um único peso e bias. Isso corresponde às dimensões da nossa `linear_layer` acima.   

Os pesos são inicializados com valores aleatórios, portanto, vamos defini-los com alguns valores conhecidos.

In [ ]:
set_w = np.array([[200]])
set_b = np.array([100])

# set_weights usa uma lista de matrizes numéricas
linear_layer.set_weights([set_w, set_b])
print(linear_layer.get_weights())

Vamos comparar a equação (1) com a saída da camada.

In [ ]:
a1 = linear_layer(X_train[0].reshape(1,1))
print(a1)
alin = np.dot(set_w,X_train[0].reshape(1,1)) + set_b
print(alin)

Eles produzem os mesmos valores!
Agora, podemos usar nossa camada linear para fazer previsões em nossos dados de treinamento.

In [ ]:
prediction_tf = linear_layer(X_train)
prediction_np = np.dot( X_train, set_w) + set_b

In [ ]:
plt_linear(X_train, Y_train, prediction_tf, prediction_np)

## Neurônio com ativação Sigmoid
A função implementada por um neurônio/unidade com uma ativação sigmoide é a mesma da regressão logística:
$$ f_{\mathbf{w},b}(x^{(i)}) = g(\mathbf{w}x^{(i)} + b) \tag{2}$$
onde $$g(x) = sigmoid(x)$$ 

Vamos definir $w$ e $b$ para alguns valores conhecidos e verificar o modelo.


### Conjunto de Dados
Usaremos agora um exemplo de regressão logística.

In [ ]:
X_train = np.array([0., 1, 2, 3, 4, 5], dtype=np.float32).reshape(-1,1)  # Matriz 2-D
Y_train = np.array([0,  0, 0, 1, 1, 1], dtype=np.float32).reshape(-1,1)  # Matriz 2-D 

In [ ]:
pos = Y_train == 1
neg = Y_train == 0
X_train[pos]

In [ ]:
pos = Y_train == 1
neg = Y_train == 0

fig,ax = plt.subplots(1,1,figsize=(4,3))
ax.scatter(X_train[pos], Y_train[pos], marker='x', s=80, c = 'red', label="y=1")
ax.scatter(X_train[neg], Y_train[neg], marker='o', s=100, label="y=0", facecolors='none', 
              edgecolors=dlc["dlblue"],lw=3)

ax.set_ylim(-0.08,1.1)
ax.set_ylabel('y', fontsize=12)
ax.set_xlabel('x', fontsize=12)
ax.set_title('gráfico de uma variável')
ax.legend(fontsize=12)
plt.show()

### Neurônio Logístico
Podemos implementar um "neurônio logístico" adicionando uma ativação sigmoide. A função do neurônio é então descrita por (2) acima.   
Esta seção criará um modelo do Tensorflow que contém nossa camada logística para demonstrar um método alternativo de criação de modelos. O Tensorflow é usado com mais frequência para criar modelos de várias camadas. O modelo [Sequential] (https://keras.io/guides/sequential_model/) é um meio conveniente de construir esses modelos.

In [ ]:
model = Sequential(
    [
        tf.keras.layers.Dense(1, input_dim=1,  activation = 'sigmoid', name='L1')
    ]
)

O `model.summary()` mostra as camadas e o número de parâmetros no modelo. Há apenas uma camada nesse modelo e essa camada tem apenas uma unidade. A unidade tem dois parâmetros, $w$ e $b$.

In [ ]:
model.summary()

In [ ]:
logistic_layer = model.get_layer('L1')
w,b = logistic_layer.get_weights()
print(w,b)
print(w.shape,b.shape)

Vamos definir o peso e a polarização para alguns valores conhecidos.

In [ ]:
set_w = np.array([[2]])
set_b = np.array([-4.5])
# set_weights usa uma lista de numpy arrays
logistic_layer.set_weights([set_w, set_b])
print(logistic_layer.get_weights())

Vamos comparar a equação (2) com a saída da camada.

In [ ]:
a1 = model.predict(X_train[0].reshape(1,1))
print(a1)
alog = sigmoidnp(np.dot(set_w,X_train[0].reshape(1,1)) + set_b)
print(alog)

Eles produzem os mesmos valores!
Agora, podemos usar nossa camada logística e o modelo NumPy para fazer previsões em nossos dados de treinamento.

In [ ]:
plt_logistic(X_train, Y_train, model, set_w, set_b, pos, neg)

O sombreamento acima reflete a saída do sigmoide, que varia de 0 a 1.

# Parabéns!
Você criou uma rede neural muito simples e explorou as semelhanças de um neurônio com a regressão linear e logística.